In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
from LocalPath import LOCALPATH
from DBSCANSupport import DBSCANSupport
from sklearn.cluster import DBSCAN

In [2]:
grav = pd.read_csv(LOCALPATH + 'data/test_curv_32.highpass.csv', names=['Longitude', 'Latitude', 'Intensity'], header=0)
grav = grav[['Longitude', 'Latitude', 'Intensity']]
Z = DBSCANSupport.formatData(grav, 'Intensity')
data = Z
testdev = DBSCANSupport(LOCALPATH+"data/sample_mask.txt.xlsx", train_zone=(-6, -1, -98, -90))
testdev.addTrainingData(data)

/Users/martinhawks/Code/Git_repos/stable_seamount/seamount_picking/code/_SeamountSupport.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["TrueSeamount"] = np.zeros(data.shape[0])


In [4]:
output = testdev.gridSearch((0.01,), (3,), verbose=True)

Score for 0.01 and 3 is -0.8775510204081632 with 33 clusters
Best score: -0.8775510204081632 with parameters (0.01, 3)


In [ ]:
print(testdev.num_seamounts)
print(testdev.seamount_points)